In [3]:
#Установка нужных пакетов
#!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

In [4]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

File ‘./quora.txt’ already there; not retrieving.


In [5]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [6]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [7]:
data_tok = []# YOUR CODE HERE

for i in range(len(data)):
    lowerWord = data[i].lower()
    data_tok.append(tokenizer.tokenize(lowerWord))

In [8]:
data_tok = []# YOUR CODE HERE

for i in range(len(data)):
    lowerWord = data[i].lower()
    data_tok.append(tokenizer.tokenize(lowerWord))

#checking

assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [9]:
data_tok[1]

['what',
 'are',
 'some',
 'ways',
 'to',
 'overcome',
 'a',
 'fast',
 'food',
 'addiction',
 '?']

###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data

In [10]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 

import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

data_tok_lemma = []

for i in range(len(data_tok)):
    for j in range(len(data_tok[i])):
        data_tok_lemma.append(lemmatizer.lemmatize(data_tok[i][j]))

[nltk_data] Downloading package omw-1.4 to /home/mors/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mors/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
from collections import Counter

print(Counter(data_tok_lemma).most_common(10))

[('?', 552413), ('the', 252068), ('what', 214798), ('is', 185392), ('a', 172513), ('i', 149873), ('to', 141788), ('in', 139813), ('how', 135687), ('of', 112001)]


###Задание 3: Подсчитайте количество разных слов до и после лемматизации

In [12]:
import numpy

data_tok_unique = numpy.unique(data_tok)
data_tok_lemma_unique = numpy.unique(data_tok_lemma)

print(len(data_tok_unique))
print(len(data_tok_lemma_unique))

/home/mors/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


536935
80303


###Задание 4: Подсчитайте количество разных слов до и после стемминга

In [13]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

data_tok_stem = []# YOUR CODE HERE

for i in range(len(data_tok)):
    for j in range(len(data_tok[i])):
        data_tok_stem.append(stemmer.stem(data_tok[i][j]))

In [14]:
data_tok_stem_unique = numpy.unique(data_tok_stem)

print(len(data_tok_unique))
print(len(data_tok_stem_unique))

536935
67026


###Задание 5: Подсчитайте количество разных слов



In [15]:
import numpy

data_unique = numpy.unique(data)

print(len(data_unique))

537251


REGEXP

https://www.programiz.com/python-programming/regex 

https://docs.python.org/3/howto/regex.html

In [16]:
import re

pattern = 'a*s'
test_string = 'abyss'
result = re.match(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")	

Search unsuccessful.


###Задание 6: 
https://www.hackerrank.com/challenges/matching-specific-string/problem?isFullScreen=true 

###Задание 7: 
https://www.hackerrank.com/challenges/matching-whitespace-non-whitespace-character/problem?isFullScreen=true

###Задание 8: 
https://www.hackerrank.com/challenges/matching-start-end/problem?isFullScreen=true

###Задание 9: 
https://www.hackerrank.com/challenges/matching-word-boundaries/problem?isFullScreen=true

In [17]:
###Задание 6 https://www.hackerrank.com/challenges/matching-specific-string/problem?isFullScreen=true 

Regex_Pattern = r'hackerrank'	# Do not delete 'r'.

###Задание 7 https://www.hackerrank.com/challenges/matching-whitespace-non-whitespace-character/problem?isFullScreen=true

Regex_Pattern = r"\S{2}\s{1}\S{2}\s{1}\S{2}"

## На занятии успел досюда :) 

##Задание 8: https://www.hackerrank.com/challenges/matching-start-end/problem?isFullScreen=true

Regex_Pattern = r"^\d{1}\w{4}.{1}$"	

## Задание 9: https://www.hackerrank.com/challenges/matching-word-boundaries/problem?isFullScreen=true

Regex_Pattern = r'\b[aeiouAEIOU]{1}[a-zA-Z]*\b'

## Redex на занятии

Regex_Pattern = '\w{2}\d+[0-1]{1}.{1}$'

Bag Of Words (BOW)

In [18]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector

def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

#Write the sentences in the corpus,in our case, just two 
string1="Welcome to Great Learning , Now start learning"
string2="Learning is a good practice"

#convert them to lower case
string1=string1.lower()
string2=string2.lower()

#split the sentences into tokens
tokens1=string1.split()
tokens2=string2.split()
print(tokens1)
print(tokens2)

#create a vocabulary list
vocab=unique(tokens1+tokens2)
print(vocab)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in stopwords and w not in special_char: 
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
vector1=vectorize(tokens1)
print(vector1)
vector2=vectorize(tokens2)
print(vector2)

['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'learning']
['learning', 'is', 'a', 'good', 'practice']
['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'is', 'a', 'good', 'practice']
['welcome', 'great', 'learning', 'now', 'start', 'good', 'practice']
[1, 1, 2, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]


Задание 10: Реализовать Bag of words на data_tok (можно на NLTK, можно без)

In [19]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector

def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

data_tok_all = []

for i in range(len(data_tok)):
    for j in range(len(data_tok[i])):
        data_tok_all.append(data_tok[i][j])

print(len(data_tok_all))

#create a vocabulary list
vocab=unique(data_tok_all)
print(len(vocab))

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in stopwords and w not in special_char: 
        filtered_vocab.append(w)
    
print(len(filtered_vocab))

vector1=vectorize(data_tok[0])
print(data_tok[0])
print(vector1[0:9])
vector2=vectorize(data_tok[1])
print(data_tok[1])
print(vector2[0:9])

7131345
87819
87811
['can', 'i', 'get', 'back', 'with', 'my', 'ex', 'even', 'though', 'she', 'is', 'pregnant', 'with', 'another', 'guy', "'", 's', 'baby', '?']
[1, 1, 1, 1, 2, 1, 1, 1, 1]
['what', 'are', 'some', 'ways', 'to', 'overcome', 'a', 'fast', 'food', 'addiction', '?']
[0, 0, 0, 0, 0, 0, 0, 0, 0]


## HomeWork ( https://regex101.com/ )

###ML1_1: 
https://www.hackerrank.com/challenges/capturing-non-capturing-groups/problem?isFullScreen=true

###ML1_2: 
https://www.hackerrank.com/challenges/branch-reset-groups/problem?isFullScreen=true

###ML1_3: 
https://www.hackerrank.com/challenges/detect-html-links/problem?isFullScreen=true

###ML1_4: Реализовать stemming, lemmatization & BoW на следующем датасете: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)

In [20]:
###ML1_1: https://www.hackerrank.com/challenges/capturing-non-capturing-groups/problem?isFullScreen=true

Regex_Pattern = r'(?:ok){3,}'

###ML1_2: https://www.hackerrank.com/challenges/branch-reset-groups/problem?isFullScreen=true

Regex_Pattern = '(^\d{2}[:]{1}\d{2}[:]{1}\d{2}[:]{1}\d{2}$)|(^\d{2}[.]{1}\d{2}[.]{1}\d{2}[.]{1}\d{2}$)|(^\d{2}[,]{1}\d{2}[,]{1}\d{2}[,]{1}\d{2}$)|(^\d{2}[-]{1}\d{2}[-]{1}\d{2}[-]{1}\d{2}$)|(^\d{2}[-]{3}\d{2}[-]{3}\d{2}[-]{3}\d{2}$)'

###ML1_3: https://www.hackerrank.com/challenges/detect-html-links/problem?isFullScreen=true

Regex_Pattern = r'<a href="(.*?)".*?>([\w ,./]*)(?=</)'

import sys, re

input()
html = sys.stdin.read()
tags =  set(re.findall("<\s*([a-z0-9]+)", html))
tags = list(tags)
tags.sort()
print(";".join(tags))

###ML1_4: Реализовать stemming, lemmatization & BoW на следующем датасете: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)

In [21]:
###ML1_4: Реализовать stemming, lemmatization & BoW на следующем датасете: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)

import pandas as pd

df = pd.read_csv('labeled.csv')

print(df['comment']) 

data = df['comment']

0                     Верблюдов-то за что? Дебилы, бл...\n
1        Хохлы, это отдушина затюканого россиянина, мол...
2                                Собаке - собачья смерть\n
3        Страницу обнови, дебил. Это тоже не оскорблени...
4        тебя не убедил 6-страничный пдф в том, что Скр...
                               ...                        
14407    Вонючий совковый скот прибежал и ноет. А вот и...
14408    А кого любить? Гоблина тупорылого что-ли? Или ...
14409    Посмотрел Утомленных солнцем 2. И оказалось, ч...
14410    КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...
14411    До сих пор пересматриваю его видео. Орамбо кст...
Name: comment, Length: 14412, dtype: object


In [22]:
# tokenization

tokenizer = WordPunctTokenizer()

data_tok = []

for i in range(len(data)):
    lowerWord = data[i].lower()
    data_tok.append(tokenizer.tokenize(lowerWord))

print(data_tok[0])


['верблюдов', '-', 'то', 'за', 'что', '?', 'дебилы', ',', 'бл', '...']


In [24]:
# stemming

from nltk.stem.snowball import SnowballStemmer 

stemmer = SnowballStemmer("russian") 

data_stem = []

for i in range(len(data_tok)):
    for j in range(len(data_tok[i])):
        data_stem.append(stemmer.stem(data_tok[i][j]))

print(data_stem[:10])

['верблюд', '-', 'то', 'за', 'что', '?', 'дебил', ',', 'бл', '...']


In [38]:
# BoW

def vectorize(tokens):
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector

def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#--------#

from nltk.corpus import stopwords

#Create lemmatizer and stopwords list
russian_stopwords = stopwords.words("russian")

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

data_tok_all = []

for i in range(len(data_tok)):
    for j in range(len(data_tok[i])):
        data_tok_all.append(data_tok[i][j])

#create a vocabulary list
vocab=unique(data_tok_all)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in russian_stopwords and w not in special_char: 
        filtered_vocab.append(w)

vector1=vectorize(data_tok[0])
print(data_tok[0])
print(vector1[:10])
vector2=vectorize(data_tok[1])
print(data_tok[1])
print(vector2[:10])

['верблюдов', '-', 'то', 'за', 'что', '?', 'дебилы', ',', 'бл', '...']
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
['хохлы', ',', 'это', 'отдушина', 'затюканого', 'россиянина', ',', 'мол', ',', 'вон', ',', 'а', 'у', 'хохлов', 'еще', 'хуже', '.', 'если', 'бы', 'хохлов', 'не', 'было', ',', 'кисель', 'их', 'бы', 'придумал', '.']
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [40]:
# Lemmas

from pymystem3 import Mystem
from string import punctuation

mystem = Mystem() 

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

print(data[10])
print(preprocess_text(data[10]))

Зачем ты пишешь хуйню, дегенерат? Поцелуй в губы ! поцелую в засос.

писать хуйня дегенерат поцелуй губа поцеловать засос
